## Problem 3: How many people live near shopping centers? (10 points)

In the last step of this analysis, use a *spatial join* to relate data from a population grid data set to the buffer layer created in *problem 2* to find out how many people live in all population grid cells that are **within** 1.5 km distance from each shopping centre. 

Use the same population grid data set as during [lesson 3](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-3/spatial-join.html) (load it directly from WFS, don’t forget to assign a CRS).


*Feel free to divide your solution into more codeblocks than prepared! Remember to add comments to your code :)*

### a) Load the population grid data set and the buffer geometries (2 points)

Use the same population grid data set as during [lesson 3](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-3/spatial-join.html) (load it directly from WFS, don’t forget to assign a CRS). Load the data into a `GeoDataFrame` called `population_grid`.

(optional) If you want, discard unneeded columns and translate the remaining column names from Finnish to English.

In [2]:
import geopandas as gpd
import pathlib

DATA_DIRECTORY = pathlib.Path().resolve()

population_grid = gpd.read_file(
    (
        "https://kartta.hsy.fi/geoserver/wfs"
        "?service=wfs"
        "&version=2.0.0"
        "&request=GetFeature"
        "&typeName=asuminen_ja_maankaytto:Vaestotietoruudukko_2020"
        "&srsName=EPSG:3879"
    ),
)
population_grid.crs = "EPSG:3879"  # for WFS data, the CRS needs to be specified manually

In [3]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
import geopandas
import pyproj

assert isinstance(population_grid, geopandas.GeoDataFrame)
assert population_grid.crs == pyproj.CRS("EPSG:3879")



Load the buffers computed in *problem 2* into a `GeoDataFrame` called `shopping_centre_buffers`. Add an `assert` statement to check whether the two data frames are in the same CRS.

In [6]:
shopping_centre_buffers = gpd.read_file(DATA_DIRECTORY / 'shopping_centres.gpkg', layer='buffers')
assert shopping_centre_buffers.crs == population_grid.crs

In [ ]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
assert isinstance(shopping_centre_buffers, geopandas.GeoDataFrame)
assert shopping_centre_buffers.geometry.geom_type.unique() == ["Polygon"]
assert shopping_centre_buffers.crs == pyproj.CRS("EPSG:3879")


---

### b) Carry out a *spatial join* between the `population_grid` and the `shopping_centre_buffers`  (4 points)

Join the shopping centre’s `id` column (and others, if you want) to the population grid data frame, for all population grid cells that are **within** the buffer area of each shopping centre. [Use a *join-type* that retains only rows from both input data frames for which the geometric predicate is true](https://geopandas.org/en/stable/gallery/spatial_joins.html#Types-of-spatial-joins). 


In [12]:
import rasterio
import geopandas as gpd
from shapely.geometry import Point
import numpy as np


raster_path = DATA_DIRECTORY / "gbr_ppp_2020_constrained.tif"  # update path if needed

with rasterio.open(raster_path) as src:
    band = src.read(1)  # read first band (population)
    transform = src.transform

    rows, cols = np.where(band > 0)  # filter out empty pixels (zero or nodata)
    values = band[rows, cols]

    xs, ys = rasterio.transform.xy(transform, rows, cols)
    points = [Point(x, y) for x, y in zip(xs, ys)]

# Create GeoDataFrame
gdf = gpd.GeoDataFrame({'population': values}, geometry=points, crs=src.crs)

In [25]:
population_grid = gdf

population_grid = population_grid.to_crs("EPSG:3879")
assert population_grid.crs == shopping_centre_buffers.crs

In [26]:
population_grid_within_buffers = population_grid.sjoin(
    shopping_centre_buffers,
    predicate='within',
    how='right'
)

population_grid_within_buffers


,index_left,population,id,name,addr,address,geometry
0,3760109.0,8.463194,0,SAINSBURY CANTERBURY,"KINGSMEAD ROAD,CANTERBURY,CT1 1BW","Kingsmead Road, The King's Mile, Canterbury, K...","POLYGON ((23847899.071 5960950.164, 23847900.8..."
0,3761898.0,1.363203,0,SAINSBURY CANTERBURY,"KINGSMEAD ROAD,CANTERBURY,CT1 1BW","Kingsmead Road, The King's Mile, Canterbury, K...","POLYGON ((23847899.071 5960950.164, 23847900.8..."
0,3761899.0,1.281360,0,SAINSBURY CANTERBURY,"KINGSMEAD ROAD,CANTERBURY,CT1 1BW","Kingsmead Road, The King's Mile, Canterbury, K...","POLYGON ((23847899.071 5960950.164, 23847900.8..."
0,3762823.0,1.568519,0,SAINSBURY CANTERBURY,"KINGSMEAD ROAD,CANTERBURY,CT1 1BW","Kingsmead Road, The King's Mile, Canterbury, K...","POLYGON ((23847899.071 5960950.164, 23847900.8..."
0,3762824.0,1.384375,0,SAINSBURY CANTERBURY,"KINGSMEAD ROAD,CANTERBURY,CT1 1BW","Kingsmead Road, The King's Mile, Canterbury, K...","POLYGON ((23847899.071 5960950.164, 23847900.8..."
...,...,...,...,...,...,...,...
528,2650986.0,49.823257,528,SAINSBURY 53 BLACKPOLE,"WINDERMERE DRIVE,WORCESTER,WR4 9JN","Windermere Drive, Lyppard Grange, Blackpole, W...","POLYGON ((23665364.454 6143628.773, 23665366.2..."
528,2650987.0,21.426699,528,SAINSBURY 53 BLACKPOLE,"WINDERMERE DRIVE,WORCESTER,WR4 9JN","Windermere Drive, Lyppard Grange, Blackpole, W...","POLYGON ((23665364.454 6143628.773, 23665366.2..."
528,2650988.0,2.599715,528,SAINSBURY 53 BLACKPOLE,"WINDERMERE DRIVE,WORCESTER,WR4 9JN","Windermere Drive, Lyppard Grange, Blackpole, W...","POLYGON ((23665364.454 6143628.773, 23665366.2..."
528,2650989.0,3.866624,528,SAINSBURY 53 BLACKPOLE,"WINDERMERE DRIVE,WORCESTER,WR4 9JN","Windermere Drive, Lyppard Grange, Blackpole, W...","POLYGON ((23665364.454 6143628.773, 23665366.2..."



---

### c) Compute the population sum around shopping centres (4 points)

Group the resulting (joint) data frame by shopping centre (`id` or `name`), and calculate the `sum()` of the population living inside the 1.5 km radius around them.

Print the results, for instance, in the form "12345 people live within 1.5 km from REDI".

In [35]:
population_sums = population_grid_within_buffers.groupby(['id', 'name', 'addr']).sum('population')
population_sums.sort_values('population',ascending=False)

,,,index_left,population
id,name,addr,,
335,SAINSBURY 4022 ESSEX ROAD,"NEW RIVER GREEN,ISLINGTON,N1 8LX",3.543980e+10,853912.5000
336,SAINSBURY 4044 UPPER ST L,"317-318 UPPER STREET,LONDON,N1 2XQ",3.533998e+10,812870.6875
16,SAINSBURYS DALSTON,"KINGSLAND SHOPPING,LONDON,E8 2LX",3.493879e+10,811147.7500
172,SAINSBURYS FULHAM BROADWA,"UNIT 17,LONDON,SW6 1BW",3.549916e+10,795833.2500
327,SAINSBURY 500 ISLINGTON,"LIVERPOOL ROAD,LONDON,N1 0RW",3.527733e+10,795607.7500
...,...,...,...,...
44,SAINSBURY ST.LEONARD,"(BRANCH NO. 0027),ST LEONARDS-ON-S,TN37 7SQ",0.000000e+00,0.0000
508,SAINSBURY 640 KIDDERMINST,"CARPET WAY,WORCS,DY11 6XP",0.000000e+00,0.0000
503,SAINSBURY 829 WOLVERHAMPT,"ST GEORGES PARADE,WEST MIDLANDS,WV2 1AY",0.000000e+00,0.0000



---

### d) Reflection

Good job! You are almost done with this week’s exercise. Please quickly answer the following short questions:
    
- How challenging did you find problems 1-3 (on scale to 1-5), and why?
- What was easy?
- What was difficult?

Add your answers in a new *Markdown* cell below: